# Why Is Your Region Hard To Count?

### By Benjamin Livingston, [*NewsCounts*](http://newscounts.org)

### April 9, 2020

# Introduction

"Hard to count" is a popular buzzphrase as the 2020 US Census takes shape - but what does it mean for your region?

As a journalist, you probably have a great feel for your area's demographic makeup, and which groups *might* be particularly hard-to-count - but it's difficult to quantify.

*NewsCounts* has built a workflow that will allow you to do this - **and we're prepared to tailor it for your newsroom using both 2010 and up-to-date 2020 data.** It's an easy, compelling template for a story on demographics and census response rates.

It's worth noting that the way the public engages with the census has changed significantly from 2010 to 2020, between the [addition of an online response option in 2020](https://2020census.gov/en/ways-to-respond.html), [the outbreak of COVID-19](https://www.census.gov/newsroom/press-releases/2020/update-on-2020-census-field-operations.html), and the evolution of the nation and its government over the last ten years.

By compiling demographic and response data from the 2010 and 2020 censuses, we have developed an idea of what makes various areas hard-to-count, and are prepared to tailor this analysis for your newsroom and its audience.

## What We'll Do Here

We'll start by examining which factors made the nation as a whole hard-to-count in 2010, and then compare it to what appears to be happening in 2020.

To do this, we will look at basic tract-level data from the 2010 decennial census and five-year American Community Survey estimates from 2010 and 2018, and use them as a means of modeling exactly *why* America's 50 states (plus the District of Columbia) seemed to be hard-to-count in 2010, and seem to be hard-to-count in 2020.

We'll then move to a more granular analysis that demostrates how the story might change in your local area, and how you can tell that story.

Unfortunately, we won't be able to say whether response rates are generally higher or lower overall quite yet, as this year's census is far from complete.

However, the census' new daily response data is extremely valuable for articles. If you need that up-to-date response rate information to pair with this, there's information on obtaining it [at the end of this guide](#Contact-Info-&amp;-Other-Resources).

## Technical Details

### Modeling

To accomplish our goal, we used two highly interpretable models that allow us to glean which demographic factors correlate most with counting difficulties:
* A basic linear model that simply calculates the linear correlation between each demographic factor and undercounting
* A simple tree model that finds which factors have the biggest impact in determining why areas are hard to count

We'll then present the weights from these models to determine which factors are most important - first the correlation coefficients for the linear model, then the variable importances from the tree model.

### Mathematical Nuts & Bolts

**Correlations** estimate how much response rates increase (positive correlation) or decrease (negative correlation) based on the increased presence of a certain subgroup. They range on a constant scale from -1 (perfectly negatively correlated), to 0 (absolutely no linear correlation), to 1 (perfectly positively correlated). These are just linear (Pearson) correlations, so they only measure linear relationships - it's possible a relationship could exist even if we don't conclude one does here.

Generally, a correlation would need to be at or above .3 or at or below -.3 for us to consider it anything more than weak - but given that most of the groups we're referring to are minorities, we can likely lower that threshold to account for their limited ability to impact response rates as a whole. If, say, a subgroup makes up only 30% of the population, it's unreasonable to expect it to have a drastic impact on overall response rates, and a correlation of .2 may be quite telling (more on this in the caveats section below).

Another way to think about correlations that is **great** for articles is this: if you square them, you get the percent of the variation in response rates that can be explained by the presence of that subgroup alone (again assuming that simple linear model).

For instance, we'll later see that there's a positive 0.72 correlation between presence of people making over \\$75,000 and census response rates in Maryland. This means that 52\% (over half!) of the variation in tract response rates in Maryland could be predicted based simply on the proportion of people making over \\$75,000 in a given tract.

[**Feature importances**](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.feature_importances_) indicate how important a demographic factor is towards determining how hard-to-count an area is relative to other factors. They place a higher priority on weighting the most important factors - unlike correlations, which treat the factors independently.

**Bootstrap p-values** estimate the probability that we would see the correlations we got in 2020 if nothing had changed since 2010. If they're zero (or close to zero), it suggests that the relationship between prevalence of that group and census response rates has changed since 2010. The *lower* the p-value, the more significant the change. We'll use a 95% level of significance with a two-sided alternative hypothesis.

Technically, our p-values tell us whether or not to reject the null hypothesis that there was no change in a correlation, not actually say that there was a definitive increase or decrease - but for simplicity, we'll label our results as "increase", "decrease", or "no change."

### Statistical Caveats

Before we begin, it's very important to note that these results can partially be traced to the relative sizes of certain subgroups in tracts. Larger subgroups will have naturally have a bigger impact on response rates, so comparing the magnitudes of these results across subgroups isn't particularly useful.

For instance, the impacts of the presence of American Indian / Alaska Native populations seem small, but they are actually difficult to ignore: they are surprisingly large given that this group has a national population of less than 5 million. Later, we will see how these impacts grow in an area with a larger AIAN population.

Also, because we have weighted all tracts equally, this analysis will inherently put more emphasis on the characteristics of communities in less populous tracts. However, approximately 95% of census tracts have a population between 1,400 and 7,300, so we don't believe this will introduce extreme bias into our data.

Finally, we are using ACS estimates from 2018 for our 2020 results, and a mix of decennial census numbers and ACS estimates for 2010; this is the best data we have for each period, but it admittedly isn't gospel truth, and the 2020 numbers are a little more of an estimation than the 2010 numbers.

# Results

We now present the factors that correlated most with undercounting in 2010. The code blocks simply pull and reform the decennial census and ACS data to fit our means.

In [1]:
# import packages
import pandas as pd
import numpy as np
import math as m
import requests
import json
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

# set API key 
key = '2988f01f5e86175bda8beae2b5035e1ccef2d052'

# tested shortcut for obtaining state FIPS codes
url = f"https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&key={key}&for=state:*"
JSONContent = requests.get(url).json()
states = pd.DataFrame(JSONContent)
states = states.iloc[1:,2]
states = [int(i) for i in states if i !='72']
states = sorted(states)

# pull 2010 mail response rates for each state and concatenate
columns=['GEO_ID','FSRR2010']
responses = pd.DataFrame(columns=columns)
for i in states:
    if i < 10:
        url = f"https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&key={key}&for=tract:*&in=state:0"\
        + str(i)
    else:
        url = f"https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&key={key}&for=tract:*&in=state:"\
        + str(i)       
    JSONContent = requests.get(url).json()        
    temp = pd.DataFrame(JSONContent)
    temp = temp.iloc[1:,[0,1]]
    temp.columns = columns
    responses = pd.concat([responses,temp])

# get 2010 decennial census numbers
url_base = f'https://api.census.gov/data/2010/dec/sf1?get=\
GEO_ID,P001001,P002005,P003002,P003003,P003004,\
P003005,P003006,P003007,P003008,P004003&key={key}&for=tract:*&in=state:'
url = url_base + '01'
JSONContent = requests.get(url).json()
temp = pd.DataFrame(JSONContent)
columns = temp.iloc[0,:-3]
dec_demos = pd.DataFrame(columns = columns)
for i in states:
    if i<10:
        url = url_base + '0' + str(i)
    else:
        url = url_base + str(i)

    JSONContent = requests.get(url).json()
    temp = pd.DataFrame(JSONContent)
    temp = temp.iloc[1:,:-3]
    temp.columns=columns
    dec_demos = pd.concat([dec_demos,temp])

# get 2010 five-year ACS averages to fill in data gaps
url_base = f'https://api.census.gov/data/2010/acs/acs5?get=GEO_ID,\
B05001_006E,B06007_033E,\
B06009_002E,B17001_002E,B06010_011E&key={key}&for=tract:*&in=state:'
url = url_base + '01'
JSONContent = requests.get(url).json()
temp = pd.DataFrame(JSONContent)
columns = temp.iloc[0,:-3]
acs_demos = pd.DataFrame(columns = columns)
for i in states:
    if i<10:
        url = url_base + '0' + str(i)
    else:
        url = url_base + str(i)
    
    JSONContent = requests.get(url).json()   
    temp = pd.DataFrame(JSONContent)
    temp = temp.iloc[1:,:-3]
    temp.columns=columns
    acs_demos = pd.concat([acs_demos,temp])

# set GEO_ID as index
dec_demos.index = dec_demos.GEO_ID
dec_demos = dec_demos.drop(columns='GEO_ID')
acs_demos.index = acs_demos.GEO_ID
acs_demos = acs_demos.drop(columns='GEO_ID')
responses.index = responses.GEO_ID
responses = responses.drop(columns='GEO_ID')

# merge response rates, ACS, and decennial data
data = responses.merge(dec_demos,how='inner',left_index=True,right_index=True)
data = data.merge(acs_demos,how='inner',left_index=True,right_index=True)
data = data[data.FSRR2010.notna()]

# rename columns
data.columns = ['response','population','rural','white',\
                'black','AIAN','asian','pacific_islander',\
                'other_race','multiracial','hispanic','noncitizen',\
               'foreign_born','no_high_school',\
               'below_poverty_level','make_over_75k']

# change data to float
data = data.astype('float32')

# remove zero population areas
data = data[data.population > 0]

# normalize by population
for column in data.columns:
    if column not in ['population','response']:
        data[column] = data[column]/data['population']

# pull data from 2018 ACS
url_base = 'https://api.census.gov/data/2018/acs/acs5?get=B01003_001E,B02001_002E,B02001_003E,B02017_001E,'\
        +'B02001_005E,B02001_006E,B02001_007E,B02001_008E,B03002_012E,B05001_006E,B99051_005E,'\
        +f'B17001_002E,B07010_011E,B06009_002E,GEO_ID&key={key}&for=tract:*&in=state:'
url = url_base + '01'
JSONContent = requests.get(url).json()
temp = pd.DataFrame(JSONContent)
columns = temp.iloc[0,:]
data_2020 = pd.DataFrame(columns=columns)

for i in states:
    if i < 10:
        url = url_base + '0' + str(i)
    else:
        url = url_base + str(i)
    try:
        JSONContent = requests.get(url).json()
        temp = pd.DataFrame(JSONContent)
        temp.columns = temp.iloc[0]
        temp = temp.iloc[1:,:]
        data_2020 = pd.concat([data_2020,temp])
    except json.JSONDecodeError:
        pass
    
# rename columns
data_2020.columns = ['population','white','black','AIAN','asian','pacific_islander',\
'other_race','multiracial','hispanic','noncitizen','foreign_born','below_poverty_level',\
'make_over_75k','no_high_school','GEO_ID','county','state','tract']

# set GEO_ID as index
data_2020.index=data_2020.GEO_ID

# drop extra columns
data_2020 = data_2020.drop(columns=['county','state','tract','GEO_ID'])

# parse population to float
data_2020.population = data_2020.population.astype('float')

# remove zero population areas
data_2020 = data_2020[data_2020.population > 0]

# parse demographic data to float and normalize by population
for column in data_2020.columns:
    if column is not 'population':
        data_2020[column] = data_2020[column].astype('float')
        data_2020[column] = data_2020[column] / data_2020['population']

# data frame to hold tract responses
tract_responses = pd.DataFrame(columns=['GEO_ID','CRRALL'])

# pull tract response data for 2020
for i in states:
    if i < 10:
        url = f"https://api.census.gov/data/2020/dec/responserate?get=GEO_ID,CRRALL&key={key}&for=tract:*&in=state:0"\
        + str(i)
    else:
        url = f"https://api.census.gov/data/2020/dec/responserate?get=GEO_ID,CRRALL&key={key}&for=tract:*&in=state:"\
        + str(i)
    try:
        JSONContent = requests.get(url).json()
        temp = pd.DataFrame(JSONContent).iloc[:,0:2]
        temp.columns = temp.iloc[0]
        temp = temp.iloc[1:,:]
        tract_responses = pd.concat([tract_responses,temp],sort=True)
    except json.JSONDecodeError:
        pass

# set index and column title for 2020 response rates
tract_responses['CRRALL'] = tract_responses['CRRALL'].astype('float')
tract_responses.index = tract_responses.GEO_ID
tract_responses = tract_responses.drop(columns = 'GEO_ID')
tract_responses.columns = ['response']

# merge demographic data and response data
data_2020 = pd.merge(data_2020,tract_responses,'inner','GEO_ID')

# create copy of original 2010 data
original_data = data.copy()

# use proportion of rural population from 2010 to approximate the same for 2020 
data_2020['rural'] = original_data['rural']

# eliminate 2020 tracts that do not match tracts from 2010
data_2020 = data_2020[pd.notnull(data_2020.rural)]

# remove five tracts that had null values for two variables
data_2020 = data_2020[pd.notnull(data_2020.make_over_75k)] 
data_2020 = data_2020[pd.notnull(data_2020.below_poverty_level)] 

# create copy of original 2020 data
original_data_2020 = data_2020.copy()

# create dataset for 2010
X_train = original_data.drop('response',axis=1)
y_train = original_data.response
                         
# fit tree model for 2010
tree_2010 = DecisionTreeRegressor()
tree_2010.fit(X_train,y_train)
 
# generate correlations & tree importances for 2010
correlations_2010 = list(original_data.corr()['response'].drop(index='response'))
tree_importances_2010 = list(tree_2010.feature_importances_)

# create dataset for 2020
X_train = original_data_2020.drop('response',axis=1)
y_train = original_data_2020.response

# fit tree model for 2020
tree_2020 = DecisionTreeRegressor()
tree_2020.fit(X_train,y_train)

# generate correlations & tree importances for 2010
correlations_2020 = list(original_data_2020.corr()['response'].drop(index='response'))
tree_importances_2020 = list(tree_2020.feature_importances_)

# concatenate results from 2010
results_2010 = pd.DataFrame(data=[correlations_2010,tree_importances_2010],\
             columns=list(data.drop('response',axis=1).columns),\
             index=['correlations_2010','tree_importances_2010'])
results_2010 = results_2010.transpose().round(2)

# concatenate results from 2020
results_2020 = pd.DataFrame(data=[correlations_2020,tree_importances_2020],\
             columns=list(data_2020.drop('response',axis=1).columns),\
             index=['correlations_2020','tree_importances_2020'])
results_2020 = results_2020.transpose().round(2)

# add dummy column
dummy_col = pd.Series(['---------------------'] * results_2020.shape[0],index=results_2010.index,name='---------------------')

# assemble full results
full_results = pd.merge(results_2010,dummy_col,'inner',left_index=True,right_index=True)
full_results = pd.merge(full_results,results_2020,'inner',left_index=True,right_index=True)
full_results = full_results.iloc[:,[0,3,2,1,4]]
full_results = full_results.reindex(full_results.correlations_2010.abs().sort_values(ascending=False).index)

bootstrapped_correlations = np.zeros((1000,original_data.shape[1]-1))
for i in range(1000):
    bootstrapped_correlations[i] = list(original_data.sample(frac=1,replace=True).corr()['response'].drop(index='response'))
bootstrap = pd.DataFrame(bootstrapped_correlations,columns = original_data.corr()['response'].drop(index='response').index)

correlations_2020 = original_data_2020.corr()['response'].drop(index='response')

pvals = pd.Series(name='Bootstrap P-value of Change in Correlation From 2010 to 2020')
for index in correlations_2020.index:
    pvals[index] = np.mean(bootstrap[index].to_numpy() <= correlations_2020[index])
p_results = pvals.copy()
p_results = p_results.astype('str')
p_results[pvals<.025] = 'Decreased'
p_results[pvals>.975] = 'Increased'
pvals.loc[pvals>.5] = 1 - pvals.loc[pvals>.5]
p_results[pvals>=.025] = 'No Change'

pvals_df = pd.DataFrame(pvals)
pvals_df['Conclusion From P-Value'] = p_results
pvals_df = pvals_df.reindex(full_results.correlations_2010.abs().sort_values(ascending=False).index)

display(full_results)
display(pvals_df)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
below_poverty_level,-0.46,-0.53,---------------------,0.27,0.33
make_over_75k,0.38,0.48,---------------------,0.08,0.09
no_high_school,-0.37,-0.51,---------------------,0.05,0.10
white,0.36,0.30,---------------------,0.07,0.05
black,-0.34,-0.26,---------------------,0.06,0.03
population,0.26,0.26,---------------------,0.10,0.08
other_race,-0.19,-0.26,---------------------,0.04,0.02
noncitizen,-0.16,-0.26,---------------------,0.04,0.04
hispanic,-0.16,-0.29,---------------------,0.06,0.05
AIAN,-0.15,-0.18,---------------------,0.05,0.04


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
below_poverty_level,0.000,Decreased
make_over_75k,0.000,Increased
no_high_school,0.000,Decreased
white,0.000,Decreased
black,0.000,Increased
population,0.416,No Change
other_race,0.000,Decreased
hispanic,0.000,Decreased
noncitizen,0.000,Decreased
AIAN,0.000,Decreased


### Findings

Examining the national results, it is abundantly clear across the board that socioeconomic factors correlate heavily with undercounting.

We find a fairly substantial positive correlation between higher income and a higher likelihood of filling out a census form, and the white- and Asian-heavy areas are filling out census forms more frequently than areas with more African-Americans, Hispanics, and native populations.

More populous, urban areas had higher return rates, as did areas with more minors.

While things appear to have changed significantly for nearly every group since 2010, it appears that Hispanic- and non citizen-heavy areas are replying at a noticeably lower clip in 2020, which many feared after the White House's attempt to [add a citizenship question](https://www.nytimes.com/2019/07/02/us/trump-census-citizenship-question.html) to this year's census.

It also appears that areas with fewer high-school educated residents and more impoverished residents tend to respond at a lower rate in 2020 than they did in 2010.

Meanwhile, it appears areas that are black-heavy are still having noticeable issues with undercounting, but this data suggests that perhaps those issues are, on the average, not quite as bad as in 2010. 

Our tree model identified economic status and education as the most important factors in determining responses - but it's possible that these are simply proxies for more granular socioeconomic differences.

We're only scratching the surface, though...

# How Do I Use This In My Newsroom?

It's **very** easy to take this analysis and adapt it for your local area.

We'll show a few examples of particularly interesting cases here. 

**Keep in mind: we can help you run data for your own state or county in literally a matter of seconds** - see the end of this article for contact details (all we have to do is add one line of code!).

To do this, we'll pretend we're in local newsrooms in a few different parts of the country.

## Primary Case Study: Kansas

Let's say we're a newsroom in Topeka or Dodge City, and do this analysis for Kansas as a whole.

In [2]:
# function to filter down to results to one geographic area using numeric fips code
def show_area(fips):
    data = original_data.copy()
    data['GEO_ID'] = data.index
    data = data[data['GEO_ID'].str.contains('1400000US' + str(fips))]
    data = data.drop(columns='GEO_ID')
    state_2010 = data.copy()

    data = original_data_2020.copy()
    data['GEO_ID'] = data.index
    data = data[data['GEO_ID'].str.contains('1400000US' + str(fips))]
    data = data.drop(columns='GEO_ID')
    state_2020 = data.copy()

    # follow same steps as original analysis

    X_train = state_2010.drop('response',axis=1)
    y_train = state_2010.response

    tree_2010 = DecisionTreeRegressor()
    tree_2010.fit(X_train,y_train)

    correlations_2010 = list(state_2010.corr()['response'].drop(index='response'))
    tree_importances_2010 = list(tree_2010.feature_importances_)

    X_train = state_2020.drop('response',axis=1)
    y_train = state_2020.response

    tree_2020 = DecisionTreeRegressor()
    tree_2020.fit(X_train,y_train)

    correlations_2020 = list(state_2020.corr()['response'].drop(index='response'))
    tree_importances_2020 = list(tree_2020.feature_importances_)

    results_2010 = pd.DataFrame(data=[correlations_2010,tree_importances_2010],\
                 columns=list(state_2010.drop('response',axis=1).columns),\
                 index=['correlations_2010','tree_importances_2010'])

    results_2010 = results_2010.transpose().round(2)

    results_2020 = pd.DataFrame(data=[correlations_2020,tree_importances_2020],\
                 columns=list(state_2020.drop('response',axis=1).columns),\
                 index=['correlations_2020','tree_importances_2020'])

    results_2020 = results_2020.transpose().round(2)

    dummy_col = pd.Series(['---------------------'] * results_2020.shape[0],index=results_2010.index,name='---------------------')

    full_results = pd.merge(results_2010,dummy_col,'inner',left_index=True,right_index=True)
    full_results = pd.merge(full_results,results_2020,'inner',left_index=True,right_index=True)
    full_results = full_results.iloc[:,[0,3,2,1,4]]
    full_results = full_results.reindex(full_results.correlations_2010.abs().sort_values(ascending=False).index)

    bootstrapped_correlations = np.zeros((1000,state_2010.shape[1]-1))
    for i in range(1000):
        bootstrapped_correlations[i] = list(state_2010.sample(frac=1,replace=True).corr()['response'].drop(index='response'))
    bootstrap = pd.DataFrame(bootstrapped_correlations,columns = original_data.corr()['response'].drop(index='response').index)

    correlations_2020 = state_2020.corr()['response'].drop(index='response')
    pvals = pd.Series(name='Bootstrap P-value of Change in Correlation From 2010 to 2020')
    for index in correlations_2020.index:
        pvals[index] = np.mean(bootstrap[index].to_numpy() <= correlations_2020[index])
    p_results = pvals.copy()
    p_results = p_results.astype('str')
    p_results[pvals<.025] = 'Decreased'
    p_results[pvals>.975] = 'Increased'
    pvals.loc[pvals>.5] = 1 - pvals.loc[pvals>.5]
    p_results[pvals>=.025] = 'No Change'

    pvals_df = pd.DataFrame(pvals)
    pvals_df['Conclusion From P-Value'] = p_results
    pvals_df = pvals_df.reindex(full_results.correlations_2010.abs().sort_values(ascending=False).index)

    display(full_results)
    display(pvals_df)
    print('Total Number of Tracts Examined: ',min(state_2010.shape[0],state_2020.shape[0]))

show_area(20)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
make_over_75k,0.55,0.65,---------------------,0.29,0.39
below_poverty_level,-0.50,-0.60,---------------------,0.10,0.14
no_high_school,-0.46,-0.60,---------------------,0.01,0.04
hispanic,-0.46,-0.48,---------------------,0.02,0.03
white,0.43,0.38,---------------------,0.11,0.07
other_race,-0.40,-0.40,---------------------,0.05,0.03
population,0.37,0.36,---------------------,0.23,0.05
noncitizen,-0.34,-0.42,---------------------,0.02,0.09
black,-0.34,-0.29,---------------------,0.02,0.02
multiracial,-0.31,-0.23,---------------------,0.01,0.03


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
make_over_75k,0.000,Increased
below_poverty_level,0.054,No Change
hispanic,0.266,No Change
no_high_school,0.006,Decreased
white,0.177,No Change
other_race,0.474,No Change
population,0.392,No Change
black,0.155,No Change
noncitizen,0.049,No Change
multiracial,0.084,No Change


Total Number of Tracts Examined:  662


Here, we see socioeconomic differences having an even bigger impact. There's a massive divide in response rates between the rich and the poor areas, and the majority-heavy and minority-heavy communities. It does not appear that these divides have been assuaged at all since 2010.

It's pretty clear that Kansas' Hispanic and [extremely diverse](https://www.thekansan.com/news/20200119/students-flourish-in-garden-city-high-school-esl-classes) minority, foreign-born, and undocumented communities are much more frequently undercounted than its more affluent, white communities.

This is a [well-documented](https://www.dodgeglobe.com/news/20200207/we-dont-exist-in-push-for-dodge-city-complete-count-census-fears-threaten-resources) issue in Kansas - but this kind of analysis can give new weight to a news story about why a region is hard-to-count by adding mathematical evidence.

Looking at the tree importances, we see an even clearer line in the sand between the rich and the poor than we saw at the national level. It seems that these disparities are even more pronounced in Kansas.

One other difference between Kansas and the United States as a whole is that we generally do not have enough evidence to say these relationships have changed significantly since 2010 in Kansas. This could be viewed as a negative in light of the divides not lessening despite the state's [complete count efforts](https://kansascounts.org/), or a positive in terms of the divides not significantly worsening despite the current political climate.

Let's continue with some more quick analyses for some other interesting areas.

## Other Quick Examples

### Florida

In [3]:
show_area(12)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
below_poverty_level,-0.34,-0.48,---------------------,0.17,0.30
population,0.26,0.23,---------------------,0.16,0.12
white,0.25,0.24,---------------------,0.05,0.04
black,-0.24,-0.24,---------------------,0.05,0.06
no_high_school,-0.21,-0.41,---------------------,0.05,0.06
other_race,-0.20,-0.19,---------------------,0.04,0.02
make_over_75k,0.16,0.24,---------------------,0.06,0.07
asian,0.16,0.19,---------------------,0.08,0.04
AIAN,-0.14,-0.07,---------------------,0.04,0.04
rural,-0.13,-0.17,---------------------,0.01,0.03


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
below_poverty_level,0.000,Decreased
population,0.074,No Change
white,0.279,No Change
black,0.384,No Change
no_high_school,0.000,Decreased
other_race,0.295,No Change
asian,0.062,No Change
make_over_75k,0.000,Increased
AIAN,0.000,Increased
rural,0.035,No Change


Total Number of Tracts Examined:  3232


In Florida, we see that response rates seem to have generally worsened significantly in more impoverished, poorly-educated areas, in areas with many non-citizens, and in areas with larger Hispanic populations. 

It's possible that these drops are tied together, and the extreme downturn we've seen in response rates among heavily Hispanic and non-citizen areas are contributing to the economic divides. 

Perhaps this could inspire a further investigation into how the current political climate is affecting the relationship between Florida's Hispanic population and the census.

### Illinois

In [4]:
show_area(17)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
below_poverty_level,-0.71,-0.67,---------------------,0.20,0.51
white,0.70,0.55,---------------------,0.49,0.07
black,-0.60,-0.43,---------------------,0.02,0.02
no_high_school,-0.54,-0.62,---------------------,0.03,0.03
make_over_75k,0.48,0.52,---------------------,0.05,0.09
other_race,-0.32,-0.42,---------------------,0.03,0.02
AIAN,-0.30,-0.14,---------------------,0.04,0.01
hispanic,-0.29,-0.43,---------------------,0.02,0.08
population,0.29,0.29,---------------------,0.03,0.05
noncitizen,-0.25,-0.36,---------------------,0.02,0.02


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
below_poverty_level,0.002,Increased
white,0.000,Decreased
black,0.000,Increased
no_high_school,0.000,Decreased
make_over_75k,0.001,Increased
other_race,0.000,Decreased
AIAN,0.000,Increased
population,0.467,No Change
hispanic,0.000,Decreased
noncitizen,0.000,Decreased


Total Number of Tracts Examined:  2967


In Illinois, we see absolutely massive divides between whites and African-Americans. However, we can also see an apparent improvement from 2010 to 2020, suggesting that perhaps these gaps in response rates have been mediated slightly. This is not the case for more heavily Hispanic areas, though - just as we would expect after looking at the rest of the country.

We can dig deeper by looking specifically at Cook County, roughly half of whose population makes up the city of Chicago.

### Cook County, IL (Chicago and surrounding areas)

In [5]:
show_area(17031)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
below_poverty_level,-0.73,-0.67,---------------------,0.61,0.52
white,0.70,0.52,---------------------,0.10,0.02
black,-0.56,-0.34,---------------------,0.01,0.03
make_over_75k,0.52,0.54,---------------------,0.06,0.11
no_high_school,-0.50,-0.64,---------------------,0.02,0.03
population,0.32,0.28,---------------------,0.04,0.05
AIAN,-0.23,-0.12,---------------------,0.02,0.01
other_race,-0.22,-0.43,---------------------,0.03,0.02
asian,0.19,0.20,---------------------,0.03,0.02
hispanic,-0.17,-0.44,---------------------,0.01,0.13


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
below_poverty_level,0.001,Increased
white,0.000,Decreased
black,0.000,Increased
make_over_75k,0.250,No Change
no_high_school,0.000,Decreased
population,0.041,No Change
AIAN,0.000,Increased
other_race,0.000,Decreased
asian,0.290,No Change
hispanic,0.000,Decreased


Total Number of Tracts Examined:  1284


We see much of the same here, which isn't surprising as Cook County makes up nearly half the population of Illinois.

### Los Angeles County

In [6]:
show_area('06037')

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
below_poverty_level,-0.49,-0.60,---------------------,0.26,0.39
noncitizen,-0.36,-0.57,---------------------,0.08,0.09
other_race,-0.36,-0.48,---------------------,0.03,0.03
no_high_school,-0.33,-0.54,---------------------,0.04,0.02
asian,0.27,0.35,---------------------,0.08,0.09
hispanic,-0.26,-0.49,---------------------,0.03,0.05
black,-0.26,-0.11,---------------------,0.10,0.03
white,0.25,0.16,---------------------,0.02,0.02
AIAN,-0.23,-0.10,---------------------,0.02,0.01
foreign_born,-0.21,-0.28,---------------------,0.03,0.03


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
below_poverty_level,0.000,Decreased
other_race,0.000,Decreased
noncitizen,0.000,Decreased
no_high_school,0.000,Decreased
asian,0.000,Increased
black,0.000,Increased
hispanic,0.000,Decreased
white,0.002,Decreased
AIAN,0.000,Increased
foreign_born,0.057,No Change


Total Number of Tracts Examined:  1997


Los Angeles County tells a similar story - the divide between whites and African-Americans has lessened, and response rates among heavier Hispanic and non-citizen communities have plummeted. More heavily Asian communities, meanwhile, appear to be responding well above the typical clip.

This could again be an outstanding jumping off point for a story about the relationship between the Hispanic community and the census, and how it has been affected by the current political climate.

### Louisiana

In [7]:
show_area(22)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
white,0.53,0.04,---------------------,0.27,0.04
black,-0.50,-0.03,---------------------,0.01,0.06
population,0.45,0.31,---------------------,0.27,0.15
make_over_75k,0.44,0.27,---------------------,0.04,0.09
below_poverty_level,-0.39,-0.32,---------------------,0.08,0.25
no_high_school,-0.21,-0.37,---------------------,0.03,0.07
other_race,-0.19,-0.03,---------------------,0.04,0.01
multiracial,-0.14,-0.12,---------------------,0.06,0.06
noncitizen,-0.11,-0.04,---------------------,0.02,0.04
hispanic,-0.11,0.01,---------------------,0.05,0.03


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
white,0.000,Decreased
black,0.000,Increased
population,0.000,Decreased
make_over_75k,0.000,Decreased
below_poverty_level,0.030,No Change
no_high_school,0.000,Decreased
other_race,0.000,Increased
multiracial,0.283,No Change
hispanic,0.001,Increased
noncitizen,0.032,No Change


Total Number of Tracts Examined:  890


Louisiana has a curious, almost unbelievable trend: the disparities in response rates between the heavier-white and heavier-black communities seem to have almost completely disappeared.

This is an extremely odd change, and it's difficult to make sense of. There are nearly 900 tracts in Louisiana, so it doesn't appear to be variation due to sample size. There's no immediate, obvious reason why this might be happening. It could be a bizarre random variation, but perhaps something else is going on here.

It would be a highly interesting trend for a newsroom to investigate.

### Maryland

In [8]:
show_area(24)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
make_over_75k,0.61,0.72,---------------------,0.38,0.53
below_poverty_level,-0.57,-0.70,---------------------,0.14,0.19
no_high_school,-0.51,-0.58,---------------------,0.05,0.03
black,-0.40,-0.51,---------------------,0.05,0.04
white,0.39,0.47,---------------------,0.03,0.03
asian,0.26,0.33,---------------------,0.05,0.02
AIAN,-0.25,-0.08,---------------------,0.03,0.01
population,0.23,0.27,---------------------,0.06,0.05
other_race,-0.20,-0.20,---------------------,0.03,0.02
hispanic,-0.14,-0.15,---------------------,0.03,0.01


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
make_over_75k,0.000,Increased
below_poverty_level,0.000,Decreased
no_high_school,0.000,Decreased
black,0.000,Decreased
white,0.002,Increased
asian,0.003,Increased
AIAN,0.000,Increased
population,0.093,No Change
other_race,0.475,No Change
hispanic,0.394,No Change


Total Number of Tracts Examined:  1266


In Maryland, we see massive, worsening divides between the rich and the poor, the educated and the less-educated, as well as most of the typical racial divides - although similarly to Los Angeles County, the more heavily Asian communities appear to be faring better than the norm.

We can look at Baltimore City to zero in on an area with very well-known disparities.

### Baltimore 

In [9]:
show_area(24510)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
below_poverty_level,-0.51,-0.67,---------------------,0.37,0.49
make_over_75k,0.42,0.66,---------------------,0.09,0.06
no_high_school,-0.40,-0.58,---------------------,0.14,0.04
population,0.35,0.26,---------------------,0.09,0.07
white,0.34,0.60,---------------------,0.06,0.13
black,-0.28,-0.55,---------------------,0.02,0.02
other_race,-0.23,-0.14,---------------------,0.04,0.02
hispanic,-0.18,-0.06,---------------------,0.01,0.01
AIAN,-0.16,-0.05,---------------------,0.03,0.03
asian,0.08,0.18,---------------------,0.03,0.02


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
below_poverty_level,0.003,Decreased
make_over_75k,0.000,Increased
no_high_school,0.001,Decreased
population,0.068,No Change
white,0.000,Increased
black,0.000,Decreased
other_race,0.103,No Change
hispanic,0.028,No Change
AIAN,0.071,No Change
asian,0.105,No Change


Total Number of Tracts Examined:  196


White this is a very small sample of tracts, it certaily appears that the rich/poor and white/African-American divides in response rates are growing far worse in Baltimore.

### Montana

In [10]:
show_area(30)

,correlations_2010,correlations_2020,---------------------,tree_importances_2010,tree_importances_2020
rural,-0.54,-0.71,---------------------,0.37,0.60
population,0.49,0.58,---------------------,0.06,0.06
AIAN,-0.37,-0.28,---------------------,0.00,0.05
white,0.33,0.24,---------------------,0.19,0.01
asian,0.23,0.21,---------------------,0.01,0.00
make_over_75k,-0.19,0.20,---------------------,0.10,0.02
black,0.19,0.21,---------------------,0.01,0.01
no_high_school,-0.17,-0.27,---------------------,0.02,0.05
multiracial,0.14,0.11,---------------------,0.04,0.02
hispanic,0.11,0.05,---------------------,0.01,0.05


,Bootstrap P-value of Change in Correlation From 2010 to 2020,Conclusion From P-Value
rural,0.000,Decreased
population,0.026,No Change
AIAN,0.306,No Change
white,0.295,No Change
asian,0.374,No Change
black,0.410,No Change
make_over_75k,0.062,No Change
no_high_school,0.077,No Change
multiracial,0.287,No Change
hispanic,0.200,No Change


Total Number of Tracts Examined:  208


In Montana, we see populous areas faring much better than rural areas, and white/black/Asian/Hispanic-heavy areas faring much better than the native-heavy areas. 

While we're again dealing with a small sample size, this does a great job of illustrating the challenge of counting native populations.

### Other States, Counties, etc.

See the section below!

# How We Can Help

*NewsCounts* can help you run this analysis for your state (or possibly even county if it's large enough), and determine which factors correlated with undercounting there. 

We can do this by looking back at 2010, and go in-the-moment and look at why it might be happening right now in 2020, and whether things are getting better or worse.

It is *very* easy for us to pop out these same numbers for just about any level of US geography with a FIPS code. We can do it almost instantaneously - just ask!

We're also quite open to incorporating new data. There are thousands of data points we can add if you feel another demographic factor might be good for modeling why your region is hard-to-count. Drop us a note if you have an idea, and we'll make an analysis that works for you.

### Running This Analysis For Yourself (if you want to try!)

We're very happy to do this for you, but if you prefer getting your hands on this tool yourself (and you're familiar with Jupyter), just follow these steps and you should be able to replicate this for your area (or multiple areas) in just minutes:
1. Download the notebook from [this project's repo](https://github.com/bwliv/censusresponses/blob/master/undercount_modeling.ipynb) by hitting "raw" and then "save as", or fork the repo and run the notebook from there
2. Run all this notebook's cells as-is (they should take about five minutes to run)
3. Add and run a block with the code *show_area(--fips--)*, where --fips-- is the FIPS code of the [state](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696) or [county](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697) you want to look up.
4. Voilà! You have your results.

### Contact Info & Other Resources

Feel free to email [benjamin.livingston@columbia.edu](mailto:benjamin.livingston@columbia.edu) or post on the NewsCounts Slack channel (email Benjamin for an invite!) any time if you'd like us to help you get started with this.

The Census Bureau is tracking 2020 response rates and provides [a wonderful map with up-to-date data](https://2020census.gov/en/response-rates.html). *NewsCounts* also provides a [beta dashboard](https://newscounts.appspot.com/) and [API](https://bernacho.github.io/NewsCounts-ResponseRatesAPI/API_documentation.html) that allows you to grab the daily response data for yourself.

[CUNY's hard-to-count map](https://www.censushardtocountmaps2020.us/) is an excellent resource for a general look back at which areas were hard-to-count in 2010.

Please don't hesistate to reach out with any census reporting-related questions. We recognize this spring is a challenging time for journalists, and we're here to make covering this pivotal census easier for you.